In [1]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 5.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617786 sha256=c0a72548b8e64021460b7cd12ab898875c154c0899c688688a72655fc6193505
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [18]:
import pickle

In [2]:
def load_data():
    import pandas as pd

    beers_key = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/beers_key.csv', index_col=0)
    prediction_table = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/beer_ids_for_prediction.csv', index_col=0)
    test = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/test.csv', index_col=0)
    train = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/train.csv', index_col=0)

    return beers_key,prediction_table,test,train

In [4]:
beers_key,prediction_table,test,train = load_data()

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [12]:
from surprise.model_selection.search import GridSearchCV
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import Reader
from surprise import Dataset

In [14]:
params = {
    'n_factors': [10, 50, 100, 150, 200],
    'n_epochs': [5, 10, 20, 50],
    'lr_all': [0.001, 0.005, 0.01],
    'reg_all':[0.01, 0.02, 0.03]
}

reader = Reader(rating_scale=(1, 10))
suprise_train = Dataset.load_from_df(train, reader)
full_trainset = suprise_train.build_full_trainset()

gs = GridSearchCV(SVD, params, measures=['rmse', 'mae'], cv=5)

gs.fit(suprise_train)

In [17]:
# best RMSE score
print(gs.best_score['rmse'])


# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.2082713522433652
{'n_factors': 10, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.03}


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
pickle.dump(gs, open('/content/drive/tuned_model.pickle', 'wb'))

OSError: ignored

In [24]:
def recommend(algo, user):
    '''
    Parameters:
        pre-fit model
        username
    Returns:
        Top 5 recommended beers for that user
    '''
    
    prediction_table['user'] = user
    
    for idx in prediction_table.index:
        prediction_table.loc[idx, ('predicted_rating')] = algo.predict(user, prediction_table['beer_id'][idx])[3]

    top_5 = prediction_table.sort_values(by='predicted_rating', ascending=False)[0:5]['beer_id']
    results = pd.DataFrame()
    for beer_id in top_5:
        results = results.append(beers_key[beers_key['beer_beerid']==beer_id])

    return results
    
    # fit algo on train set (surprise trainset object)
    # add user to beer_ids_for_prediction
    # run algo.predict on beer_ids_for_prediction
    # sort results, return top 5 beerids
    # translate beer ids to beer names
    # return beer names

In [26]:
recommend(gs, 'changeup45')

ValueError: ignored